In [1]:
LOCAL_MODE = True

# 0. 환경설정

In [1]:
import argparse
import os
import requests
import tempfile
import subprocess, sys

import pandas as pd
import numpy as np
from glob import glob
import copy
from collections import OrderedDict
from pathlib import Path
import joblib

import logging
import logging.handlers

import json
import base64
import boto3
import sagemaker
from botocore.client import Config
from botocore.exceptions import ClientError

import time
from datetime import datetime as dt
import datetime
from pytz import timezone
from dateutil.relativedelta import *

In [2]:
# 한국 시간
KST = dt.today() + relativedelta(hours=9)
print(f"Start job time: {KST}")

Start job time: 2022-11-28 18:10:41.966503


In [3]:
def get_secret():
    # See https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
    secret_name = "prod/sagemaker"
    region_name = "ap-northeast-2"
    
    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )
    try:
        get_secret_value_response = client.get_secret_value(
            SecretId='prod/sagemaker',
        )
    except ClientError as e:
        if e.response['Error']['Code'] == 'DecryptionFailureException': # Secrets Manager can't decrypt the protected secret text using the provided KMS key.
            raise e
        elif e.response['Error']['Code'] == 'InternalServiceErrorException': # An error occurred on the server side.
            raise e
        elif e.response['Error']['Code'] == 'InvalidParameterException': # You provided an invalid value for a parameter.
            raise e
        elif e.response['Error']['Code'] == 'InvalidRequestException': # You provided a parameter value that is not valid for the current state of the resource.
            raise e
        elif e.response['Error']['Code'] == 'ResourceNotFoundException': # We can't find the resource that you asked for.
            raise e
    else:
        if 'SecretString' in get_secret_value_response:
            secret = get_secret_value_response['SecretString']
            return secret
        else:
            decoded_binary_secret = base64.b64decode(get_secret_value_response['SecretBinary'])
            return decoded_binary_secret

keychain = json.loads(get_secret())
ACCESS_KEY_ID = keychain['ACCESS_KEY_ID_ent']
ACCESS_SECRET_KEY = keychain['ACCESS_SECRET_KEY_ent']

BUCKET_NAME_USECASE = keychain['BUCKET_NAME_USECASE_ent']
S3_PATH_STAGE = keychain['S3_PATH_STAGE']
S3_PATH_GOLDEN = keychain['S3_PATH_GOLDEN']
S3_PATH_TRAIN = keychain['S3_PATH_TRAIN']
S3_PATH_log = keychain['S3_PATH_LOG']
S3_PATH_FORECAST = keychain['S3_PATH_FORECAST']

boto3_session = boto3.Session(ACCESS_KEY_ID, ACCESS_SECRET_KEY)
sm_session = sagemaker.Session(boto_session = boto3_session)
region = boto3_session.region_name

s3_resource = boto3_session.resource('s3')
bucket = s3_resource.Bucket(BUCKET_NAME_USECASE)
s3_client = boto3_session.client('s3')
sm_client = boto3.client('sagemaker',
                         aws_access_key_id = ACCESS_KEY_ID,
                         aws_secret_access_key = ACCESS_SECRET_KEY,
                         region_name = 'ap-northeast-2')

In [19]:
%%writefile src/prediction.py

import argparse
import os
import requests
import tempfile
import subprocess, sys
import json

import glob
import pandas as pd
import joblib
import pickle
import tarfile
from io import StringIO, BytesIO

import logging
import logging.handlers

import time
from datetime import datetime as dt

import boto3


###############################
######### util 함수 설정 ##########
###############################
def _get_logger():
    '''
    로깅을 위해 파이썬 로거를 사용
    # https://stackoverflow.com/questions/17745914/python-logging-module-is-printing-lines-multiple-times
    '''
    loglevel = logging.DEBUG
    l = logging.getLogger(__name__)
    if not l.hasHandlers():
        l.setLevel(loglevel)
        logging.getLogger().addHandler(logging.StreamHandler(sys.stdout))        
        l.handler_set = True
    return l  
logger = _get_logger()


def get_bucket_key_from_uri(uri):
    uri_aws_path = uri.split('//')[1]
    uri_bucket = uri_aws_path.rsplit('/')[0]
    uri_file_path = '/'.join(uri_aws_path.rsplit('/')[1:])
    return uri_bucket, uri_file_path

def get_secret():
    # See https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
    secret_name = "prod/sagemaker"
    region_name = "ap-northeast-2"
    
    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )
    try:
        get_secret_value_response = client.get_secret_value(
            SecretId='prod/sagemaker',
        )
    except ClientError as e:
        if e.response['Error']['Code'] == 'DecryptionFailureException': # Secrets Manager can't decrypt the protected secret text using the provided KMS key.
            raise e
        elif e.response['Error']['Code'] == 'InternalServiceErrorException': # An error occurred on the server side.
            raise e
        elif e.response['Error']['Code'] == 'InvalidParameterException': # You provided an invalid value for a parameter.
            raise e
        elif e.response['Error']['Code'] == 'InvalidRequestException': # You provided a parameter value that is not valid for the current state of the resource.
            raise e
        elif e.response['Error']['Code'] == 'ResourceNotFoundException': # We can't find the resource that you asked for.
            raise e
    else:
        if 'SecretString' in get_secret_value_response:
            secret = get_secret_value_response['SecretString']
            return secret
        else:
            decoded_binary_secret = base64.b64decode(get_secret_value_response['SecretBinary'])
            return decoded_binary_secret
        
if __name__=='__main__':
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'autogluon==0.6.0'])
    from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

    ############################################
    ###### Secret Manager에서 키값 가져오기  #######
    ########################################### 
    logger.info(f"\n### Loading Key value from Secret Manager")
    
    keychain = json.loads(get_secret())
    ACCESS_KEY_ID = keychain['ACCESS_KEY_ID_ent']
    ACCESS_SECRET_KEY = keychain['ACCESS_SECRET_KEY_ent']

    BUCKET_NAME_USECASE = keychain['BUCKET_NAME_USECASE_ent']
    S3_PATH_STAGE = keychain['S3_PATH_STAGE']
    S3_PATH_GOLDEN = keychain['S3_PATH_GOLDEN']
    S3_PATH_TRAIN = keychain['S3_PATH_TRAIN']
    S3_PATH_log = keychain['S3_PATH_LOG']
    boto3_session = boto3.Session(ACCESS_KEY_ID, ACCESS_SECRET_KEY)

    region = boto3_session.region_name

    s3_resource = boto3_session.resource('s3')
    s3_client = boto3_session.client('s3')
    sm_client = boto3.client('sagemaker',
                             aws_access_key_id = ACCESS_KEY_ID,
                             aws_secret_access_key = ACCESS_SECRET_KEY,
                             region_name = 'ap-northeast-2')
    
    ################################
    ###### 커맨드 인자 파싱   ##########
    ################################    
    parser = argparse.ArgumentParser()
    parser.add_argument('--base_input_dir', type=str, default="/opt/ml/processing/input", help='train,testset 불러오는곳')
    parser.add_argument('--output_dir', type = str, default = "/opt/ml/processing/output", help='예측 결과값이 저장되는 곳, test dataset과 prediction 결과가 merge되서 저장된다.')
    parser.add_argument('--model_package_group_name', type=str, default='palm-oil-price-forecast')   
    args = parser.parse_args()     
    logger.info("\n######### Argument Info ####################################")
    logger.info(f"args.base_input_dir: {args.base_input_dir}")
    logger.info(f"args.output_dir: {args.output_dir}")
    logger.info(f"args.model_package_group_name: {args.model_package_group_name}")

    base_input_dir = args.base_input_dir
    output_dir = args.output_dir
    model_package_group_name = args.model_package_group_name
    model_dir = '/opt/ml/model'
    
    ##########################################################
    ###### 적합한 모델의 URI 찾고, 탑 성능 모델 이름 가져오기 ##########
    #########################################################
    logger.info("\n######### Finding suitable model uri ####################################")
    logger.info(f"Model Group name: {model_package_group_name}")
    model_registry_list = sm_client.list_model_packages(ModelPackageGroupName = model_package_group_name)['ModelPackageSummaryList']
    for model in model_registry_list:
        if (model['ModelPackageGroupName'] == model_package_group_name and
            model['ModelApprovalStatus'] == 'Approved'):
            mr_arn = model['ModelPackageArn']
            break
    describe_model = sm_client.describe_model_package(ModelPackageName=mr_arn)
    s3_model_uri = describe_model['InferenceSpecification']['Containers'][0]['ModelDataUrl']
    top_model_name = describe_model['ModelPackageDescription'].split(',')[1]

    logger.info(f"Found suitable model uri: {s3_model_uri}")
    logger.info(f"And top model name: {top_model_name}")
    
    logger.info("\n#########Download suitable model file  ####################################")
    model_bucket, model_key = get_bucket_key_from_uri(s3_model_uri)  
    model_obj = s3_client.get_object(Bucket = model_bucket, Key = model_key)
    
    ##########################################################
    ###### 모델 압축 풀고 TimeseriesDataFrame으로 변환 ##########
    #########################################################
    logger.info("\n######### Model zip file extraction ####################################")
    with tarfile.open(fileobj=model_obj['Body'], mode='r|gz') as file:
        file.extractall(model_dir)    
    logger.info(f"list in /opt/ml/model: {os.listdir(model_dir)}")        
    
    logger.info("\n######### Convert df_test dataframe into TimeSeriesDataFrame  ###########")        
    df_train = pd.read_csv(os.path.join(f'{base_input_dir}/train/train.csv'))
    df_train.loc[:, "ds"] = pd.to_datetime(df_train.loc[:, "ds"])
    tdf_train = TimeSeriesDataFrame.from_data_frame(
        df_train,
        id_column="ric",
        timestamp_column="ds",
    )
    df_test = pd.read_csv(f"{base_input_dir}/test/test.csv")
    df_test.loc[:, "ds"] = pd.to_datetime(df_test.loc[:, "ds"])
    tdf_test = TimeSeriesDataFrame.from_data_frame(
        df_test,
        id_column="ric",
        timestamp_column="ds",
    )
    logger.info(f"df_test sample: tail(2) \n {tdf_train.tail(2)}")
    logger.info(f"df_test sample: head(2) \n {tdf_test.head(2)}")
    
    ################################
    ###### Prediction 시작 ##########
    ###############################
    logger.info("\n######### Start prediction  ###########")        
    loaded_trainer = pickle.load(open(f"{model_dir}/models/trainer.pkl", 'rb'))
    logger.info(f"loaded_trainer: {loaded_trainer}")
    prediction_ag_model = loaded_trainer.predict(data = tdf_train,
                                                 model = top_model_name)
    logger.info(f"prediction_ag_model sample: head(2) \n {prediction_ag_model.head(2)}")

    prediction_result = pd.merge(tdf_test.loc['FCPOc3']['y'], prediction_ag_model.loc['FCPOc3'],
                                 left_index = True, right_index = True, how = 'left')
    prediction_result.to_csv(f'{output_dir}/prediction_result.csv')

Overwriting src/prediction.py


In [20]:
prediction_code = 'src/prediction.py'
%store prediction_code

Stored 'prediction_code' (str)


In [21]:
%store

Stored variables and their in-db values:
bucket                             -> 'palm-oil-price-forecast'
leaderboard_uri                    -> 's3://palm-oil-price-forecast/trained-model/2022/1
model_validation_code              -> 'src/model_validation.py'
prediction_code                    -> 'src/prediction.py'
preproc_data_dir                   -> 's3://palm-oil-price-forecast/golden-data/2022/11/
preprocessed_stage_uri             -> 's3://palm-oil-price-forecast/golden-data/2022/11/
preprocessed_test_uri              -> 's3://palm-oil-price-forecast/golden-data/2022/11/
preprocessed_train_uri             -> 's3://palm-oil-price-forecast/golden-data/2022/11/
preprocessing_code                 -> 'src/preprocessing.py'
project_prefix                     -> 'palm-oil-price-forecast'
stage_data_uri                     -> 's3://palm-oil-price-forecast/staged-data'
test_data_uri                      -> 's3://palm-oil-price-forecast/golden-data/2022/11/
train_data_uri                   

In [7]:
%store -r

In [9]:
df_train = pd.read_csv(train_data_uri)
df_train.tail()

,ds,high,low,open,y,ric
219403,2022-10-24,0.541064,0.557112,0.547093,0.528296,Wc3
219404,2022-10-25,0.528051,0.549802,0.534302,0.522264,Wc3
219405,2022-10-26,0.528918,0.546147,0.529360,0.531744,Wc3
219406,2022-10-27,0.541643,0.556503,0.535465,0.527722,Wc3
219407,2022-10-28,0.522556,0.544319,0.531686,0.519966,Wc3


In [10]:
df_test = pd.read_csv(test_data_uri)
df_test.head()

,ds,high,low,open,y,ric
0,2022-10-31,0.741585,0.742476,0.725275,0.735406,BOc1
1,2022-11-01,0.745962,0.746199,0.735653,0.736168,BOc1
2,2022-11-02,0.766038,0.743252,0.742827,0.772291,BOc1
3,2022-11-03,0.768302,0.759231,0.772283,0.766651,BOc1
4,2022-11-04,0.788981,0.784052,0.770299,0.797439,BOc1


# 1. 모델 빌딩 파이프라인 의 스텝(Step) 생성
## 1) 모델 빌딩 파이프라인 변수 생성
파이프라인에서 사용할 파이프라인 파라미터를 정의합니다. 파이프라인을 스케줄하고 실행할 때 파라미터를 이용하여 실행조건을 커스마이징할 수 있습니다. 파라미터를 이용하면 파이프라인 실행시마다 매번 파이프라인 정의를 수정하지 않아도 됩니다.

지원되는 파라미터 타입은 다음과 같습니다:

- ParameterString - 파이썬 타입에서 str
- ParameterInteger - 파이썬 타입에서 int
- ParameterFloat - 파이썬 타입에서 float
이들 파라미터를 정의할 때 디폴트 값을 지정할 수 있으며 파이프라인 실행시 재지정할 수도 있습니다. 지정하는 디폴트 값은 파라미터 타입과 일치하여야 합니다.

본 노트북에서 사용하는 파라미터는 다음과 같습니다.

- processing_instance_type - 프로세싱 작업에서 사용할 ml.* 인스턴스 타입
- processing_instance_count - 프로세싱 작업에서 사용할 인스턴스 개수
- training_instance_type - 학습작업에서 사용할 ml.* 인스턴스 타입
- model_approval_status - 학습된 모델을 CI/CD를 목적으로 등록할 때의 승인 상태 (디폴트는 "PendingManualApproval")
- input_data - 입력데이터에 대한 S3 버킷 URI
파이프라인의 각 스텝에서 사용할 변수를 파라미터 변수로서 정의 합니다.

## 2) 로컬에서 테스트 - 실패

In [13]:
if LOCAL_MODE:
    # 도커 컨테이너 입력 폴더: train, test data가 들어가는 부분
    base_input_dir = 'opt/ml/processing/input'
    os.makedirs(base_input_dir, exist_ok=True)

    # 도커 컨테이너 기본 출력 폴더: 예측결과가 출력되는곳
    base_output_dir = 'opt/ml/processing/output'
    os.makedirs(base_output_dir, exist_ok=True)

    model_package_group_name = bucket

In [15]:
print(train_data_uri)
print(test_data_uri)
print(base_input_dir)

s3://palm-oil-price-forecast/golden-data/2022/11/26/train.csv
s3://palm-oil-price-forecast/golden-data/2022/11/26/test.csv
opt/ml/processing/input


In [16]:
!aws s3 cp 's3://palm-oil-price-forecast/golden-data/2022/11/26/train.csv' \
            'opt/ml/processing/input/train.csv'

download: s3://palm-oil-price-forecast/golden-data/2022/11/26/train.csv to opt/ml/processing/input/train.csv


In [17]:
!aws s3 cp 's3://palm-oil-price-forecast/golden-data/2022/11/26/test.csv' \
            'opt/ml/processing/input/test.csv'

download: s3://palm-oil-price-forecast/golden-data/2022/11/26/test.csv to opt/ml/processing/input/test.csv


In [ ]:
!python src/prediction.py --base_input_path {base_input_dir} \
                          --output_dir {base_output_dir} \
                          --model_package_group_name {bucket} \

Traceback (most recent call last):
  File "src/prediction.py", line 23, in <module>
    from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
ModuleNotFoundError: No module named 'autogluon'


# 2. 파이프라인 정의 및 실행

In [8]:
from sagemaker.workflow.parameters import (ParameterInteger,
                                           ParameterString,
                                          )

prediction_instance_type = ParameterString(
    name = "PredctionInstanceType",
    default_value = "ml.m5.xlarge"
)
prediction_instance_count = ParameterInteger(
    name = "PredctionInstanceCount",
    default_value = 1
)
input_train_data = ParameterString(
    name="InputTrainData",
    default_value = train_data_uri,
)
input_test_data = ParameterString(
    name="InputTestData",
    default_value = test_data_uri,
)
# input_model_group_name = ParameterString(
#     name="modelPackageGroupName",
#     default_value = project_prefix,
# )

In [41]:
BUCKET_NAME_USECASE

'palm-oil-price-forecast'

In [12]:
prediction_output_path = f"s3://{BUCKET_NAME_USECASE}/{S3_PATH_FORECAST}/{KST.strftime('%Y/%m/%d')}"
prediction_output_path

's3://palm-oil-price-forecast/forecasted-data/2022/11/28'

## 1) 스텝정의 -실패

### (1) MXNetProcessor실패!

In [13]:
## BUG
from sagemaker.mxnet import MXNetProcessor

framework_version = '1.9.0'
py_version = 'py38'

mxnet_processor = MXNetProcessor(
    framework_version = framework_version,
    py_version = py_version,
    instance_type = 'ml.m5.xlarge',
    instance_count = 1,
    base_job_name = 'Palm-oil-forecast-Prediction',
    role = sagemaker.get_execution_role(),
)

In [15]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep
######################################################################################3
step_prediction = ProcessingStep(
    name = 'Palm_oil_forecast-Prediction-Autogluon060',
    processor = mxnet_processor,
    inputs=[
        ProcessingInput(source = train_data_uri,
                        destination = "/opt/ml/processing/input/train"),
        ProcessingInput(source = test_data_uri,
                        destination = "/opt/ml/processing/input/test"),
    ],
    outputs=[
        ProcessingOutput(output_name = "prediction_data",
                         source = "/opt/ml/processing/output",
                         destination = prediction_output_path)
        ],
    job_arguments=["--model_package_group_name", bucket],
    code = prediction_code
)

### (2) ScriptProcessor 실패!

In [23]:
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput

script_processor = ScriptProcessor(
    command=['python3'],
    image_uri=image_uri,
    instance_type = 'ml.m5.xlarge',
    instance_count = 1,
    base_job_name = 'Palm-oil-forecast-Prediction',
    role = sagemaker.get_execution_role(),
)

In [24]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep
step_prediction = ProcessingStep(
    name = 'Palm_oil_forecast-Prediction-Autogluon060',
    processor = script_processor,
    inputs=[
        ProcessingInput(source = train_data_uri,
                        destination = "/opt/ml/processing/input/train"),
        ProcessingInput(source = test_data_uri,
                        destination = "/opt/ml/processing/input/test"),
    ],
    outputs=[
        ProcessingOutput(output_name = "prediction_data",
                         source = "/opt/ml/processing/output",
                         destination = prediction_output_path)
        ],
    code = prediction_code
)

### 1) 스텝 정의

In [ ]:
# %%time
# from sagemaker.processing import ProcessingInput, ProcessingOutput
# from sagemaker.workflow.steps import ProcessingStep

# inputs = [
#     ProcessingInput(source = train_data_uri,
#                     destination = "/opt/ml/processing/input/train"),
#     ProcessingInput(source = test_data_uri,
#                     destination = "/opt/ml/processing/input/test"),
# ]

# outputs = [
#     ProcessingOutput(output_name = "prediction_data",
#                      source = "/opt/ml/processing/output",
#                      destination = prediction_output_path)
# ]

# step_args = mxnet_processor.run(
#     code='prediction.py',
#     source_dir='src',
#     inputs = inputs,
#     outputs = outputs,
#     arguments=["--model_package_group_name", input_model_group_name],
# )

# step_prediction = ProcessingStep(
#     name="Palm-oil-forecast-Prediction",
#     step_args = step_args
# )


In [ ]:
# %%time
# from sagemaker.processing import ProcessingInput, ProcessingOutput
# from sagemaker.workflow.steps import ProcessingStep

# inputs = [
#     ProcessingInput(source = train_data_uri,
#                     destination = "/opt/ml/processing/input/train"),
#     ProcessingInput(source = test_data_uri,
#                     destination = "/opt/ml/processing/input/test"),
# ]

# outputs = [
#     ProcessingOutput(output_name = "prediction_data",
#                      source = "/opt/ml/processing/output",
#                      destination = prediction_output_path)
# ]

# step_args = mxnet_processor.run(
#     code='prediction.py',
#     source_dir='src',
#     inputs = inputs,
#     outputs = outputs,
#     arguments=["--model_package_group_name", project_prefix],
# )

# step_prediction = ProcessingStep(
#     processor = mxnet_processor,
#     name = "Palm-oil-forecast-Prediction",
#     step_args = step_args,
#     job_arguments = ["--model_package_group_name", project_prefix],    
#     code = prediction_code # 추가하고 다시 진행 사유: ValueError: code None url scheme b'' is not recognized. Please pass a file path or S3 url

# )

### 1) 파이프라인 실행

In [25]:
from sagemaker.workflow.pipeline import Pipeline

pipeline_name = project_prefix
pipeline = Pipeline(name = pipeline_name,
                    parameters = [
                        prediction_instance_type,        
                        prediction_instance_count,         
                        input_train_data,
                        input_test_data,
                    ],
                    steps=[step_prediction]
)

In [26]:
import json

definition = json.loads(pipeline.definition())
definition

{'Version': '2020-12-01',
 'Metadata': {},
 'Parameters': [{'Name': 'PredctionInstanceType',
   'Type': 'String',
   'DefaultValue': 'ml.m5.xlarge'},
  {'Name': 'PredctionInstanceCount', 'Type': 'Integer', 'DefaultValue': 1},
  {'Name': 'InputTrainData',
   'Type': 'String',
   'DefaultValue': 's3://palm-oil-price-forecast/golden-data/2022/11/28/train.csv'},
  {'Name': 'InputTestData',
   'Type': 'String',
   'DefaultValue': 's3://palm-oil-price-forecast/golden-data/2022/11/28/test.csv'}],
 'PipelineExperimentConfig': {'ExperimentName': {'Get': 'Execution.PipelineName'},
  'TrialName': {'Get': 'Execution.PipelineExecutionId'}},
 'Steps': [{'Name': 'Palm_oil_forecast-Prediction-Autogluon060',
   'Type': 'Processing',
   'Arguments': {'ProcessingResources': {'ClusterConfig': {'InstanceType': 'ml.m5.xlarge',
      'InstanceCount': 1,
      'VolumeSizeInGB': 30}},
    'AppSpecification': {'ImageUri': '763104351884.dkr.ecr.ap-northeast-2.amazonaws.com/mxnet-training:1.9.0-cpu-py38',
     'C

In [ ]:
%%time
start = time.time()
pipeline.upsert(role_arn=sagemaker.get_execution_role())
execution = pipeline.start()
execution.wait()
end = time.time()

In [29]:
print(f"prediction 시간 : {end - start:.1f} sec")
print(f"prediction 시간 : {((end - start)/60):.1f} min")

prediction 시간 : 423.1 sec
prediction 시간 : 7.1 min


[2022년 11월 29일]
- prediction 시간 : 423.1 sec
- prediction 시간 : 7.1 min

In [30]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:ap-northeast-2:276114397529:pipeline/palm-oil-price-forecast',
 'PipelineExecutionArn': 'arn:aws:sagemaker:ap-northeast-2:276114397529:pipeline/palm-oil-price-forecast/execution/wb19211ha4m2',
 'PipelineExecutionDisplayName': 'execution-1669628909946',
 'PipelineExecutionStatus': 'Succeeded',
 'PipelineExperimentConfig': {'ExperimentName': 'palm-oil-price-forecast',
  'TrialName': 'wb19211ha4m2'},
 'CreationTime': datetime.datetime(2022, 11, 28, 9, 48, 29, 875000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2022, 11, 28, 9, 55, 14, 262000, tzinfo=tzlocal()),
 'CreatedBy': {},
 'LastModifiedBy': {},
 'ResponseMetadata': {'RequestId': 'fe395d12-b556-4784-8ee7-7c0a5e7f48ff',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'fe395d12-b556-4784-8ee7-7c0a5e7f48ff',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '520',
   'date': 'Mon, 28 Nov 2022 21:28:32 GMT'},
  'RetryAttempts': 0}}

In [31]:
execution.list_steps()

[{'StepName': 'Palm_oil_forecast-Prediction-Autogluon060',
  'StartTime': datetime.datetime(2022, 11, 28, 9, 48, 31, 268000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2022, 11, 28, 9, 55, 13, 709000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'AttemptCount': 0,
  'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:ap-northeast-2:276114397529:processing-job/pipelines-wb19211ha4m2-palm-oil-forecast-pr-zup0rtm4vr'}}}]

In [ ]:
model_approval_status = ParameterString(
    name="ModelApprovalStatus", default_value="PendingManualApproval"
)